In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# import function

In [ ]:
!pip install pyinform
!pip install adjustText
!pip install kmeans1d
import sys
import pandas as pd
import numpy as np
from pyinform.dist import Dist
from pyinform.shannon import entropy
from scipy import stats
from scipy.spatial.distance import squareform
import math
import csv
drive_path  = "/content/drive/Shareddrives/專題/"

     |████████████████████████████████| 133kB 5.5MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-cp37-none-any.whl size=7097 sha256=aa8ddca459a3f5613954ff95eb7bfb1a3ff39485d020bfa5ad9d5b4bd7704c75
  Stored in directory: /root/.cache/pip/wheels/41/95/74/7d347e136d672f8bc28e937032bc92baf4f80856763a7e7b72
Successfully built adjustText
     |████████████████████████████████| 102kB 4.8MB/s 


#save the following items and check
1. IM＿icd9 = [ ]
2. IM＿icd9_chinese_name = [ ]

In [ ]:
# save icd9 
# ================== modify path ==================
#icd9_path = drive_path + "CSV_files/IM_HM_icd9.csv"
#icd9_txt_path = drive_path + "txt_files/IM_HM_icd9_375.txt"
icd9_path = drive_path + "CSV_files/ICD9.csv"
icd9_txt_path = drive_path + "txt_files/ICD9_1094.txt"
# =================================================

# =================== variable ====================
All_icd9 = []
All_icd9_chinese_name = []
check_cnt_csv=0
# =================================================

# Build All_icd9 & All_icd9_chinese_name
with open(icd9_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        icd9_tmp = str(row['ICD-9-CM 2001年版']).strip()
        All_icd9.append(icd9_tmp)
        All_icd9_chinese_name.append(row['中文疾病名稱'])
        check_cnt_csv += 1

# get the disease_id_list based on ICD9.txt
disease_id_list = []
check_cnt_txt = 0
with open(icd9_txt_path, "r") as file1:
    for line in file1.readlines():
        icd9_str = str(line)
        if icd9_str[0].isdigit():
            disease_str = icd9_str.strip()
            disease_id_list.append(disease_str)  
            check_cnt_txt +=1



# check if items in ICD9.csv is ame as ICD9.txt
if (check_cnt_csv != check_cnt_txt):
  print("insonsistence between check_cnt_csv & check_cnt_txt")
print("cnt(csv) : ",check_cnt_csv)
print("cnt(txt) : ",check_cnt_txt)

for i in range(0,check_cnt_csv):
  if (All_icd9[i] != disease_id_list[i]) : print(i,All_icd9[i],disease_id_list[i])

cnt(csv) :  1094
cnt(txt) :  1094


#entropy function

In [ ]:
def entropy_i(i, sum):
    p_i = i/sum
    tmp= (-1) * p_i * (math.log(p_i, 10)) #entropy_i
    return tmp

class Pair: 
    def __init__(self,C, sumA, sumB,NAB):
        self.C = C
        self.NAB = NAB
        self. sumA=sumA
        self.sumB=sumB
        self.Z = 0.0
        self.D = 0.0
    
    #Distance(A,B)
    def Distance_AB(self):
        sigma_sum = 0
        self.Z = 0.0
        for i in range(0,self.NAB) : 
             # f_i= entropy( p1(wk) ) + entropy( p2(wk) )
             f_i = entropy_i(self.C['icd9_A_cnt'][i], sumA) + entropy_i(self.C['icd9_B_cnt'][i], sumB) 
             self.Z += f_i
             # |R1(wk)-R2(wk)|
             error_i = abs( self.C['icd9_A_rank'][i]-self.C['icd9_B_rank'][i] )
             sigma_sum += (error_i * f_i)
       
        if (self.Z == 0):
          return -1
        self.D = sigma_sum / self.NAB / self.Z
        return self.D

# Build Distance matrix   ----------- Modify parameters here!!! -----------

In [ ]:
##########################################################
#----------------------- Modify -----------------------
icd9_cnt = check_cnt_txt
No_axis = check_cnt_txt
input_path = drive_path + "ab/ab_merged/"
##########################################################

##TEST

In [ ]:
for icd9_i in range (0,icd9_cnt):
  
  icd9_tmp = All_icd9[icd9_i].strip()
  #convert xxx to xxx.csv ; xxx.x to xxxx.csv
  if len(icd9_tmp)==3 :
     fA = icd9_tmp+'.csv'
  else : 
    fA = icd9_tmp[:icd9_tmp.find('.')]+icd9_tmp[icd9_tmp.find('.')+1]+'.csv'
  fileA = input_path + fA
  try:
    with open(fileA,"r") as csvfile1:
      rows = csv.reader(csvfile1, delimiter=',')
    csvfile1.close()
  except:
    print(icd9_i ,fA)

## Method 1 : Rank is defined by 1,2,.....cnt(row)

In [ ]:
# Build distance matrix of IM_icd9 (IM:內科)
Distance_matrix = np.zeros(shape =(icd9_cnt,icd9_cnt))

for icd9_i in range (0,icd9_cnt):
  
  icd9_tmp = All_icd9[icd9_i].strip()
  #convert xxx to xxx.csv ; xxx.x to xxxx.csv
  if len(icd9_tmp)==3 :
     fA = icd9_tmp+'.csv'
  else : 
    fA = icd9_tmp[:icd9_tmp.find('.')]+icd9_tmp[icd9_tmp.find('.')+1]+'.csv'
  fileA = input_path + fA

  #-----------------------------------------------
  # Read A
  A = {}
  A_num = 0
  sumA = 0
  with open(fileA,"r") as csvfile1:
    rows = csv.reader(csvfile1, delimiter=',')
    #skip header
    next(rows, None)
    for row in rows :
      if (row[0]=='nan' or row[0]=="" or row[0]=='Nan') : continue
      A_num += 1
      a_icd = row[0].strip()
      a_cnt = (int)(row[1])
      A[a_icd] = (a_cnt,A_num) #a_cnt = 人數 ; A_num = rank
      sumA += a_cnt
  csvfile1.close()

  #-----------------------------------------------
  for icd9_j in range (0,icd9_cnt):
    if(icd9_i==icd9_j):
      Distance_matrix[icd9_i][icd9_j]=0.0
      continue
    if(icd9_i>icd9_j):
      continue

    icd9_tmp2 = All_icd9[icd9_j].strip()
    if len(icd9_tmp2)==3 : fB= icd9_tmp2+'.csv'
    else : fB= icd9_tmp2[:icd9_tmp2.find('.')]+icd9_tmp2[icd9_tmp2.find('.')+1]+'.csv'
    fileB=input_path + fB

    B = {}
    B_num = 0
    sumB = 0
    with open(fileB,"r") as csvfile2:
      rows = csv.reader(csvfile2, delimiter=',')
      #skip header
      next(rows, None)
      for row in rows:
        if (row[0]=='nan' or row[0]=="") : continue
        B_num += 1
        b_icd = row[0].strip()
        b_cnt = (int)(row[1])
        B[b_icd] = (b_cnt, B_num)
        sumB += b_cnt
    csvfile2.close()

    Aset = set(A)
    Bset = set(B)

    C_num = 0
    C = { 'icd9_id'   :[],
      'icd9_A_cnt'  :[],
      'icd9_B_cnt'  :[],
      'icd9_A_rank'  :[],
      'icd9_B_rank'  :[]
    }

    for key in Aset.intersection(Bset):
        a_cnt = A[key][0]
        b_cnt = B[key][0]
        if a_cnt > 0 and b_cnt > 0 :
          C['icd9_id'].append(key)

          C['icd9_A_cnt'].append(A[key][0])
          C['icd9_A_rank'].append(A[key][1])

          C['icd9_B_cnt'].append(B[key][0])
          C['icd9_B_rank'].append(B[key][1])

          C_num += 1
    #===========================================================================
    pair1 = Pair(C, sumA, sumB, C_num)
    Distance_pair1  = pair1.Distance_AB()

    # symmetric
    Distance_matrix[icd9_i][icd9_j]=Distance_pair1
    Distance_matrix[icd9_j][icd9_i]=Distance_pair1

    # check current icd9 of A
  print(icd9_i, icd9_tmp)

Condensed_Distance_matrix = squareform(Distance_matrix)
# 後來改名 distance_matrix.txt
np.savetxt('/content/drive/Shareddrives/專題/txt_files/result/DIST_'+ str(icd9_cnt) +'.txt', Condensed_Distance_matrix)


## Method 2 : Rank is defined by 1~200 (similar to PR)

In [ ]:
# Build distance matrix of IM_icd9 (IM:內科)
Distance_matrix = np.zeros(shape =(icd9_cnt,icd9_cnt))

for icd9_i in range (0,icd9_cnt):
  
  icd9_tmp = All_icd9[icd9_i].strip()
  #convert xxx to xxx.csv ; xxx.x to xxxx.csv
  if len(icd9_tmp)==3 :
     fA = icd9_tmp+'.csv'
  else : 
    fA = icd9_tmp[:icd9_tmp.find('.')]+icd9_tmp[icd9_tmp.find('.')+1]+'.csv'
  fileA = input_path + fA

  #-----------------------------------------------
  # Read A
  A = {}
  A_num = 0
  sumA = 0
  with open(fileA,"r") as csvfile1:
    rows = csv.reader(csvfile1, delimiter=',')
    #skip header
    next(rows, None)
    for row in rows :
      if (row[0]=='nan' or row[0]=="") : continue
      A_num += 1
      a_icd = row[0].strip()
      a_cnt = (int)(row[1])
      A[a_icd] = (a_cnt,A_num) #a_cnt = 人數 ; A_num = rank
      sumA += a_cnt
  csvfile1.close()

  #-----------------------------------------------

  for icd9_j in range (0,icd9_cnt):
    if(icd9_i==icd9_j):
      Distance_matrix[icd9_i][icd9_j]=0.0
      continue
    if(icd9_i>icd9_j):
      continue

    icd9_tmp2 = All_icd9[icd9_j].strip()
    if len(icd9_tmp2)==3 : fB= icd9_tmp2+'.csv'
    else : fB= icd9_tmp2[:icd9_tmp2.find('.')]+icd9_tmp2[icd9_tmp2.find('.')+1]+'.csv'
    fileB=input_path + fB

    B = {}
    B_num = 0
    sumB = 0
    with open(fileB,"r") as csvfile2:
      rows = csv.reader(csvfile2, delimiter=',')
      #skip header
      next(rows, None)
      for row in rows:
        if (row[0]=='nan' or row[0]=="") : continue
        B_num += 1
        b_icd = row[0].strip()
        b_cnt = (int)(row[1])
        B[b_icd] = (b_cnt, B_num)
        sumB += b_cnt
    csvfile2.close()


    Aset = set(A)
    Bset = set(B)


    C_num = 0
    C = { 'icd9_id'   :[],
      'icd9_A_cnt'  :[],
      'icd9_B_cnt'  :[],
      'icd9_A_rank'  :[],
      'icd9_B_rank'  :[]
    }

    ###### modify ###############
    A_block_size = math.floor(A_num/200)
    B_block_size = math.floor(B_num/200)
    ##############################

    for key in Aset.intersection(Bset):
        a_cnt = A[key][0]
        b_cnt = B[key][0]
        if a_cnt > 0 and b_cnt > 0 :
          C['icd9_id'].append(key)

          C['icd9_A_cnt'].append(A[key][0])
          ###### modify ###############
          a_rank = math.floor(A[key][1] / A_block_size ) +1 
          C['icd9_A_rank'].append(a_rank)
          ##############################

          C['icd9_B_cnt'].append(B[key][0])
          ###### modify ###############
          try:
            b_rank = math.floor(B[key][1] / B_block_size )+1 
          except:
            print(B[key][1])
            print(B_block_size)
          C['icd9_B_rank'].append(b_rank)
          ##############################

          C_num += 1



    #===========================================================================
    pair1 = Pair(C, sumA, sumB, C_num)
    Distance_pair1  = pair1.Distance_AB()


    # symmetric
    Distance_matrix[icd9_i][icd9_j]=Distance_pair1
    Distance_matrix[icd9_j][icd9_i]=Distance_pair1

    # check current icd9 of A
  print(icd9_i, icd9_tmp)

Condensed_Distance_matrix = squareform(Distance_matrix)
# 後來改名 distance_matrix_pr.txt
np.savetxt('/content/drive/Shareddrives/專題/txt_files/result/DIST'+ str(icd9_cnt) +'_filtered_1.txt', Condensed_Distance_matrix)


## Tool : Read Distance_matrix (check count of items)

In [ ]:
# 直接讀 Distance_matrix 
from scipy.spatial.distance import squareform
n=icd9_cnt
pre_Distance_matrix=[]
Distance_matrix = np.zeros(shape =(icd9_cnt,icd9_cnt))
cnt=0
with open('/content/drive/Shareddrives/專題/txt_files/result/DIST_1094.txt', "r") as file1:
    for line in file1.readlines():
        pre_Distance_matrix.append(float(line))
        cnt+=1

Distance_matrix=squareform(pre_Distance_matrix)
Condensed_Distance_matrix = squareform(Distance_matrix)
print(cnt)


597871
